In [12]:
# feasbility study
import pandas as pd
df = pd.read_csv("/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org.csv")
end_dev = df[df.timestamp == "01/01/2023 00:00"].index.values[0]
df = df[:end_dev]
df.to_csv("/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org_feasibility.csv")

In [41]:
import pandas as pd
df_org = pd.read_csv("/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org.csv")
df_dev = pd.read_csv("/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org_feasibility.csv")
df_org.timestamp[0]

'11/05/2022 07:30'

In [30]:
# feasbility study
import pandas as pd
df = pd.read_csv("/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Wave_Height.label.final.csv")
end_dev = df[df.timestamp == "01/09/2022 00:00"].index.values[0]
df = df[:end_dev]
df.to_csv("/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Wave_Height.label_feasibility.csv")

In [81]:
import pandas as pd
mapping_alg = pd.read_csv("/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results/mapping_algorithm.csv")
mapping_alg

,official_name,cite_key,folder
0,NoveltySVR,~\cite{ma2003online},novelty_svr
1,PS-SVM,~\cite{ma2003time},phasespace_svm
2,EmsembleGI,~\cite{gao2020ensemble},ensemble_gi
3,GrammarViz,~\cite{senin2015time},grammarviz3
4,HOT SAX,~\cite{keogh2005hot},hotsax
5,TSBitmap,~\cite{wei2005assumption},ts_bitmap
6,NormA-SJ,~\cite{boniol2021unsupervised},norma
7,SAND,~\cite{boniol2021sand},sand
8,Series2Graph,~\cite{boniol2022series2graph},series2graph
9,STAMP,~\cite{yeh2016matrix},stamp


In [107]:
THRESHOLD_INDICATING_SHUTDOWN = 5
MISSING_VALUE = -999
OPERATION_VAL_RANGE = (713.682, 763.826)

In [108]:
def remove_long_shutdown(numbers, num_consecutive, missing_label):
    chunks = []
    current_chunk = []

    i = 0
    while i < len(numbers)-1:
        num = numbers[i]
        if num != missing_label:
            current_chunk.append(i)
        else:
            j = i+1
            while j < len(numbers):
                if numbers[j] == missing_label:
                    j += 1
                else:
                    break

            if j-i < num_consecutive:
                current_chunk += range(i,min(j+1, len(numbers)))# numbers[i:j+1]
            else:
                chunks.append(current_chunk)
                current_chunk = []

            i=j         

        i+= 1

    # Append the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    to_ret = []
    # Print the chunks
    for i, chunk in enumerate(chunks, 1):
        to_ret += chunk
        
    return to_ret

In [109]:
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn import metrics
import traceback

def calculate_f1(path_to_result, label_file, data_source, feasibility=None):
    for result_file in os.listdir(path_to_result):
        if data_source in result_file and result_file.endswith(".ts"):
            alg_name = result_file[:result_file.find(data_source)]
            alg_name = '_'.join(alg_name.split('_')[1:])[:-1]
            try:
                #print('----------', result_file, '----------')
                result_alg = np.loadtxt(result_file)
                result_alg = pd.DataFrame(result_alg, columns=["alg_anomaly_score"])

                label = pd.read_csv(label_file)
                label = label[-len(result_alg):]
                label = label.reset_index(drop=True)
                
                if feasibility:
                    label_feasibility = pd.read_csv(feasibility)
                    cutoff_label_feasibility = label.timestamp[0]
                    cutoff_label_feasibility = label_feasibility[label_feasibility.timestamp == cutoff_label_feasibility].index.values[0]
                    label_feasibility = label_feasibility[cutoff_label_feasibility:]

                total = pd.concat([result_alg, label], axis=1, join='inner')
                if feasibility:
                    total = total[:len(label_feasibility)]
                observed_values = total.value.values.tolist()
                observed_values = [int(x) for x in observed_values]
                to_keep_comparision = remove_long_shutdown(observed_values, THRESHOLD_INDICATING_SHUTDOWN, MISSING_VALUE)
                total=total[total.index.isin(to_keep_comparision)]

                threshold = np.mean(total.alg_anomaly_score) + 3*np.std(total.alg_anomaly_score)
                total["alg_anomaly_labels"] = np.where(total.alg_anomaly_score >= threshold, 1, 0)

                Precision, Recall, F, Support =metrics.precision_recall_fscore_support(total.is_anomaly.values.tolist(), 
                                                                                       total.alg_anomaly_labels.values.tolist(), 
                                                                                       zero_division=0)
                precision = Precision[1]
                recall = Recall[1]
                f = F[1]
                #print(precision, recall, f)
                
                #print(alg_name)
                #print(mapping_alg[mapping_alg.folder == alg_name])
                print(mapping_alg[mapping_alg.folder == alg_name].official_name.values[0], ",", f)
            except:
                #traceback.print_exc()
                try:
                    print(mapping_alg[mapping_alg.folder == alg_name].official_name.values[0], ",", f)
                except:
                    pass#print(alg_name, ",", f)
        
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org.csv"
data_source = "Tide_Pressure"
calculate_f1(path_to_result, label_file, data_source)

Donut , 0.5217391304347826
DSPOT , 0.05555555555555555
DWT-MLEAD , 0.022222222222222223
EmsembleGI , 0.0
FFT , 0.009300417151063393
RForest , 0.5979381443298969
XGBoosting , 0.5217391304347826
GrammarViz , 0.5217391304347826
HOT SAX , 0.06741573033707865
IE-CAE , 0.0
Left STAMPi , 0.04608294930875576
MedianMethod , 0.5631067961165049
NoveltySVR , 0.04651162790697675
NumetaHTM , 0.0
PCI , 0.3717948717948718
PS-SVM , 0.022371364653243846
SARIMA , 0.08780487804878048
SR-CNN , 0.0
SR , 0.004987531172069825
Sub-IF , 0.0061162079510703364
Sub-LOF , 0.025974025974025972
S-H-ESD , 0.27848101265822783
Triple ES , 0.3373493975903614
TSBitmap , 0.013761467889908256


In [110]:
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org.csv"
feasibility = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org_feasibility.csv"
data_source = "Tide_Pressure"
calculate_f1(path_to_result, label_file, data_source, feasibility)

Donut , 0.5806451612903226
DSPOT , 0.06666666666666667
DWT-MLEAD , 0.014388489208633096
EmsembleGI , 0.0
FFT , 0.009581881533101046
RForest , 0.6865671641791045
XGBoosting , 0.5806451612903226
GrammarViz , 0.5806451612903226
HOT SAX , 0.08771929824561403
IE-CAE , 0.018691588785046728
Left STAMPi , 0.05177993527508091
MedianMethod , 0.6111111111111112
NoveltySVR , 0.05217391304347826
NumetaHTM , 0.0
PCI , 0.4516129032258064
PS-SVM , 0.03361344537815126
SARIMA , 0.06993006993006994
SR-CNN , 0.0
SR , 0.0
Sub-IF , 0.009259259259259259
Sub-LOF , 0.03773584905660377
S-H-ESD , 0.24000000000000002
Triple ES , 0.3018867924528302
TSBitmap , 0.011904761904761904


In [111]:
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn import metrics
import traceback
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def calculate_roc(path_to_result, label_file, data_source, feasibility=None):
    for result_file in os.listdir(path_to_result):
        if data_source in result_file and result_file.endswith(".ts"):
            alg_name = result_file[:result_file.find(data_source)]
            alg_name = '_'.join(alg_name.split('_')[1:])[:-1]
            try:
                #print('----------', result_file, '----------')
                result_alg = np.loadtxt(result_file)
                result_alg = pd.DataFrame(result_alg, columns=["alg_anomaly_score"])

                label = pd.read_csv(label_file)
                label = label[-len(result_alg):]
                label = label.reset_index(drop=True)
                
                if feasibility:
                    label_feasibility = pd.read_csv(feasibility)
                    cutoff_label_feasibility = label.timestamp[0]
                    cutoff_label_feasibility = label_feasibility[label_feasibility.timestamp == cutoff_label_feasibility].index.values[0]
                    label_feasibility = label_feasibility[cutoff_label_feasibility:]

                total = pd.concat([result_alg, label], axis=1, join='inner')
                if feasibility:
                    total = total[:len(label_feasibility)]

                total = pd.concat([result_alg, label], axis=1, join='inner')
                observed_values = total.value.values.tolist()
                observed_values = [int(x) for x in observed_values]
                to_keep_comparision = remove_long_shutdown(observed_values, THRESHOLD_INDICATING_SHUTDOWN, MISSING_VALUE)
                total=total[total.index.isin(to_keep_comparision)]

                roc_auc = metrics.roc_auc_score(total.is_anomaly, total.alg_anomaly_score)
                #print("Estimated ROC AUC:", roc_auc)

                y, x, _ = metrics.precision_recall_curve(total.is_anomaly, total.alg_anomaly_score)
                pr_auc = metrics.auc(x, y)
                #print("Estimated PR AUC:", pr_auc)
                
                #print(alg_name)
                #print(mapping_alg[mapping_alg.folder == alg_name].official_name.values[0])
                print(mapping_alg[mapping_alg.folder == alg_name].official_name.values[0], ",", roc_auc, ",", pr_auc)
            except:
                #traceback.print_exc()
                try:
                    print(mapping_alg[mapping_alg.folder == alg_name].official_name.values[0], ",", roc_auc, ",", pr_auc)
                except:
                    pass
        
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org.csv"
data_source = "Tide_Pressure"
calculate_roc(path_to_result, label_file, data_source)

Donut , 0.7322634858509105 , 0.4123461356973447
DSPOT , 0.5269267781662261 , 0.05792611137640613
DWT-MLEAD , 0.5147439489359629 , 0.005161612828468255
EmsembleGI , 0.4446765660084701 , 0.003995171375513696
FFT , 0.5 , 0.5023359670216421
RForest , 0.5 , 0.5023359670216421
XGBoosting , 0.5 , 0.5023359670216421
GrammarViz , 0.5 , 0.5023359670216421
HOT SAX , 0.5406408991428421 , 0.07232687832578487
IE-CAE , 0.4715649135862032 , 0.004453945505696916
Left STAMPi , 0.6054172811942341 , 0.013816145535733203
MedianMethod , 0.701191534804023 , 0.47822589817243943
NoveltySVR , 0.6482589256307226 , 0.014273361142125122
NumetaHTM , 0.4984123697107752 , 0.0023359670216420475
PCI , 0.8181711595385721 , 0.4207478798624856
PS-SVM , 0.7291608297906034 , 0.011673518959100975
SARIMA , 0.8078074054223057 , 0.28546677453895214
SR-CNN , 0.30368919653772397 , 0.0030503200802327367
SR , 0.5436100926185342 , 0.0053323944179967855
Sub-IF , 0.6016628515908042 , 0.0056830066456650735
Sub-LOF , 0.6487943384192709 

In [112]:
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org.csv"
feasibility = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Tide_Pressure.org_feasibility.csv"
data_source = "Tide_Pressure"
calculate_roc(path_to_result, label_file, data_source,feasibility)

Donut , 0.7322634858509105 , 0.4123461356973447
DSPOT , 0.5269267781662261 , 0.05792611137640613
DWT-MLEAD , 0.5147439489359629 , 0.005161612828468255
EmsembleGI , 0.4446765660084701 , 0.003995171375513696
FFT , 0.5 , 0.5023359670216421
RForest , 0.5 , 0.5023359670216421
XGBoosting , 0.5 , 0.5023359670216421
GrammarViz , 0.5 , 0.5023359670216421
HOT SAX , 0.5406408991428421 , 0.07232687832578487
IE-CAE , 0.4715649135862032 , 0.004453945505696916
Left STAMPi , 0.6054172811942341 , 0.013816145535733203
MedianMethod , 0.701191534804023 , 0.47822589817243943
NoveltySVR , 0.6482589256307226 , 0.014273361142125122
NumetaHTM , 0.4984123697107752 , 0.0023359670216420475
PCI , 0.8181711595385721 , 0.4207478798624856
PS-SVM , 0.7291608297906034 , 0.011673518959100975
SARIMA , 0.8078074054223057 , 0.28546677453895214
SR-CNN , 0.30368919653772397 , 0.0030503200802327367
SR , 0.5436100926185342 , 0.0053323944179967855
Sub-IF , 0.6016628515908042 , 0.0056830066456650735
Sub-LOF , 0.6487943384192709 

In [113]:
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Wave_Height.label.final.csv"
data_source = "Wave_Height"
OPERATION_VAL_RANGE = (0, 15.2)
calculate_f1(path_to_result, label_file, data_source)

Donut , 0.0015576323987538943
DSPOT , 0.03268945022288262
DWT-MLEAD , 0.02653631284916201
EmsembleGI , 0.0
FFT , 0.22627865961199295
RForest , 0.0015576323987538943
XGBoosting , 0.0015576323987538943
GrammarViz , 0.0015576323987538943
HOT SAX , 0.01762114537444934
IE-CAE , 0.0
Left STAMPi , 0.0
MedianMethod , 0.18630910374029638
NoveltySVR , 0.07123655913978494
NumetaHTM , 0.034353995519044056
PCI , 0.0031128404669260703
PS-SVM , 0.0
SARIMA , 0.0015576323987538943
SR-CNN , 0.014025245441795231
SR , 0.3314139475367882
Sub-IF , 0.0
Sub-LOF , 0.0031128404669260703
S-H-ESD , 0.03930635838150289
Triple ES , 0.14507042253521127
TSBitmap , 0.0209351011863224


In [114]:
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Wave_Height.label.final.csv"
feasibility = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Wave_Height.label_feasibility.csv"
data_source = "Wave_Height"
OPERATION_VAL_RANGE = (0, 15.2)
calculate_f1(path_to_result, label_file, data_source, feasibility)

Donut , 0.002306805074971165
DSPOT , 0.04736275565123789
DWT-MLEAD , 0.03861788617886179
EmsembleGI , 0.0
FFT , 0.25134233057611377
RForest , 0.002306805074971165
XGBoosting , 0.002306805074971165
GrammarViz , 0.002306805074971165
HOT SAX , 0.013186813186813187
IE-CAE , 0.0
Left STAMPi , 0.0
MedianMethod , 0.07964601769911504
NoveltySVR , 0.0712871287128713
NumetaHTM , 0.04814004376367615
PCI , 0.004608294930875576
PS-SVM , 0.0
SARIMA , 0.002306805074971165
SR-CNN , 0.008658008658008658
SR , 0.30769230769230765
Sub-IF , 0.0
Sub-LOF , 0.002306805074971165
S-H-ESD , 0.055030094582975066
Triple ES , 0.12140575079872205
TSBitmap , 0.01084598698481562


In [115]:
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Wave_Height.label.final.csv"
data_source = "Wave_Height"
OPERATION_VAL_RANGE = (0, 15.2)
calculate_roc(path_to_result, label_file, data_source)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Donut , 0.04024638058997521 , 0.0694667724399305
DSPOT , 0.5062372068968028 , 0.24586948198236805
DWT-MLEAD , 0.4828403856004091 , 0.12559126065394677
EmsembleGI , 0.48710411669424347 , 0.12119756773645625
FFT , 0.5 , 0.563786417420702
RForest , 0.5 , 0.563786417420702
XGBoosting , 0.5 , 0.563786417420702
GrammarViz , 0.5 , 0.563786417420702
HOT SAX , 0.501192098243926 , 0.1452957895986942
IE-CAE , 0.5596823971658252 , 0.1575798789721151
Left STAMPi , 0.588174162008366 , 0.17355021147772431
MedianMethod , 0.8057010891493521 , 0.6947380563243788
NoveltySVR , 0.731137079878048 , 0.5732586199676661
NumetaHTM , 0.5121031350864641 , 0.23996889824476866
PCI , 0.9851727478675125 , 0.9437716480303637
PS-SVM , 0.6055162626203224 , 0.1817158271741211
SARIMA , 0.9208401283392209 , 0.614391313211465
SR-CNN , 0.34329886522663566 , 0.09266190445030807
SR , 0.8208675955903869 , 0.7069811922553699
Sub-IF , 0.5995260615677749 , 0.19748361215578397
Sub-LOF , 0.5047391178366625 , 0.12933539635556354
S-H-

In [116]:
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Wave_Height.label.final.csv"
feasibility = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/Wave_Height.label_feasibility.csv"
data_source = "Wave_Height"
OPERATION_VAL_RANGE = (0, 15.2)
calculate_roc(path_to_result, label_file, data_source, feasibility)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Donut , 0.04024638058997521 , 0.0694667724399305
DSPOT , 0.5062372068968028 , 0.24586948198236805
DWT-MLEAD , 0.4828403856004091 , 0.12559126065394677
EmsembleGI , 0.48710411669424347 , 0.12119756773645625
FFT , 0.5 , 0.563786417420702
RForest , 0.5 , 0.563786417420702
XGBoosting , 0.5 , 0.563786417420702
GrammarViz , 0.5 , 0.563786417420702
HOT SAX , 0.501192098243926 , 0.1452957895986942
IE-CAE , 0.5596823971658252 , 0.1575798789721151
Left STAMPi , 0.588174162008366 , 0.17355021147772431
MedianMethod , 0.8057010891493521 , 0.6947380563243788
NoveltySVR , 0.731137079878048 , 0.5732586199676661
NumetaHTM , 0.5121031350864641 , 0.23996889824476866
PCI , 0.9851727478675125 , 0.9437716480303637
PS-SVM , 0.6055162626203224 , 0.1817158271741211
SARIMA , 0.9208401283392209 , 0.614391313211465
SR-CNN , 0.34329886522663566 , 0.09266190445030807
SR , 0.8208675955903869 , 0.7069811922553699
Sub-IF , 0.5995260615677749 , 0.19748361215578397
Sub-LOF , 0.5047391178366625 , 0.12933539635556354
S-H-

In [118]:
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/C3_Temperature.org.csv"
feasibility = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/C3_Temperature.org.csv"
data_source = "C3_Temperature"
OPERATION_VAL_RANGE = (-2, 32)
calculate_f1(path_to_result, label_file, data_source, feasibility)

Donut , 0.000606685676151186
DSPOT , 0.0
DWT-MLEAD , 0.087979374798582
FFT , 0.3148234889256388
RForest , 0.0
XGBoosting , 0.0
GrammarViz , 0.0
IE-CAE , 0.0
Left STAMPi , 0.0006058770069675855
MedianMethod , 0.024069028156221615
NumetaHTM , 0.0007256016446970613
PCI , 0.03313903281519862
PS-SVM , 0.05473249003297732
SR , 0.013340233454085448
SR-CNN , 0.012782522805182734
Sub-IF , 0.04722460249452537
Sub-LOF , 0.023050603277507655
S-H-ESD , 0.03655352480417755
Triple ES , 0.0007278903311901007
TSBitmap , 0.03667974133361603


In [119]:
path_to_result = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/2-results"
label_file = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/C3_Temperature.org.csv"
feasibility = "/mnt/c/Arbeid/00_Projects/AutoQC_git/anomaly_detection_algs/1-data/C3_Temperature.org.csv"
data_source = "C3_Temperature"
OPERATION_VAL_RANGE = (-2, 32)
calculate_roc(path_to_result, label_file, data_source, feasibility)

Donut , 0.6398476980646663 , 0.2802701337249267
DSPOT , 0.33195240104851564 , 0.29655476719283025
DWT-MLEAD , 0.33358835933038566 , 0.16294146769001677
FFT , 0.5 , 0.5934096478489744
RForest , 0.5 , 0.5934096478489744
XGBoosting , 0.5 , 0.5934096478489744
GrammarViz , 0.5 , 0.5934096478489744
IE-CAE , 0.6623497563649978 , 0.4815714748920167
Left STAMPi , 0.32667352172439335 , 0.13092208481709616
MedianMethod , 0.40216907668366814 , 0.1477728915438506
NumetaHTM , 0.49967552162499695 , 0.13513558215493307
PCI , 0.34403036189728287 , 0.13996165864402382
PS-SVM , 0.470995017907907 , 0.1616953277315691
SR , 0.33389116322586543 , 0.13559058160897672
SR-CNN , 0.6025397445543148 , 0.19529554220607248
Sub-IF , 0.47538153351407986 , 0.15586545717436168
Sub-LOF , 0.4171482186158944 , 0.16342668789556308
S-H-ESD , 0.49143424653962725 , 0.1604143119777641
Triple ES , 0.32009500646434097 , 0.1328075350124419
TSBitmap , 0.5726416438547483 , 0.20480206385556973
